In [21]:
from openpyxl import Workbook,load_workbook
from sage import all
from sage.groups.all import BraidGroup
from sage.knots.all import Link
B = BraidGroup(10)
wb = load_workbook("dirty_braid_data.xlsx")
ws = wb.active

In [2]:
#function to extract a list to be turned into 
#sage braid object from given data cell;
#ignores braces, commas, and whitespace;
#if any other characters are found, will return 'bad_data'
#assumes all braids have index <=10
def extract_bn(cell):
    b = []
    l = len(cell)
    i = 0
    max_iters = 1000
    if(cell[0:2] == '[{'):
        while ((i < l)and(i < max_iters)):
            if(cell[i]=='{'):
                i = i+1
            elif(cell[i:i+2]=='}]'):
                break
            elif(cell[i]==','):
                i = i+1
            elif(cell[i]==' '):
                i=i+1
            elif(cell[i]=='M'):
                i = i+1
            elif(cell[i]=='('):
                i = i+1
            elif(cell[i]==')'):
                i = i+1
            elif(cell[i:i+2]=='[{'):
                i = i+2
            elif(cell[i]=='}'):
                break
            elif(cell[i].isnumeric()):
                b.append(int(cell[i]))
                i = i+1
            elif((cell[i] =='-')and(cell[i+1:i+2].isnumeric())):
                b.append(int(cell[i:i+2]))
                i = i + 2
            else:
                return []
    else:
        while ((i < l)and(i < max_iters)):
            if(cell[i]=='{'):
                i = i+1
            elif(cell[i]==','):
                i = i+1
            elif(cell[i]==' '):
                i=i+1
            elif(cell[i]=='M'):
                i = i+1
            elif(cell[i]=='('):
                i = i+1
            elif(cell[i]==')'):
                i = i+1
            elif(cell[i]=='}'):
                i=i+1
            elif(cell[i].isnumeric()):
                b.append(int(cell[i]))
                i = i+1
            elif((cell[i] =='-')and(cell[i+1:i+2].isnumeric())):
                b.append(int(cell[i:i+2]))
                i = i + 2
            else:
                return []
    return b

In [3]:
#function to extract single braid when multiples are given
#similar to extract_bn
def extract_single_bn(cell):
    b=[]
    l = len(cell)
    i = 0
    max_iters = 1000
    while ((i < l)and(i < max_iters)):
        if(cell[i]=='{'):
            i = i+1
        elif(cell[i]=='}'):
            break
        elif(cell[i]==','):
            i = i+1
        elif(cell[i]==' '):
            i=i+1
        elif(cell[i]=='M'):
            i = i+1
        elif(cell[i]=='('):
            i = i+1
        elif(cell[i]==')'):
            i = i+1
        elif(cell[i]=='['):
            i = i+1
        elif(cell[i]==']'):
            i = i+1
        elif(cell[i].isnumeric()):
            b.append(int(cell[i]))
            i = i+1
        elif((cell[i] =='-')and(cell[i+1:i+2].isnumeric())):
            b.append(int(cell[i:i+2]))
            i = i + 2
        else:
            return []
    return b

In [4]:
#function to test whether two knots are mirror images or not
def check_mirror(L_1,L_2,index,errors):
    M_1 = L_1.mirror_image()
    M_2 = L_2.mirror_image()
    J_1 = L_1.jones_polynomial()
    J_2 = L_2.jones_polynomial()
    J_M1 = M_1.jones_polynomial()
    J_M2 = M_2.jones_polynomial()
    if(J_1 == J_2):
        if((J_1 == J_M1)or(J_2 == J_M2)):
            print("JP of knot in row {0} is equal to JP of its mirror.".format(index))
            return 'uncertain'
        else:
            return 'N'
    elif(J_1 == J_M2):
        if((J_1 == J_M1)or(J_2 == J_M2)):
            print("JP of knot in row {0} is equal to JP of its mirror.".format(index))
            return 'uncertain'
        else:
            return 'Y'
    else:
        errors.append(index)
        return 'uncertain'

In [5]:
#function to format positive braid notation
def format_pbraid(is_mirror, pbraid):
    formatted = u''
    i = 0
    l = len(pbraid)
    if(is_mirror == 'Y'):
        formatted = formatted + 'M{'
    else:
        formatted = formatted + '{'
    while((i < l) and (i < 1000)):
        if(i == 0):
            formatted = formatted + str(pbraid[i])
            i = i+1
        else:
            formatted = formatted + ',' + str(pbraid[i])
            i = i+1
    formatted = formatted + '}'
    return formatted

In [6]:
#function to format sqp braid notation
def format_sqpbraid(is_mirror, sqpbraid):
    formatted = u''
    i = 0
    l = len(sqpbraid)
    if(is_mirror == 'Y'):
        formatted = formatted + 'M{'
    else:
        formatted = formatted + '{'
    while((i < l) and (i < 1000)):
        k = 0
        if(i == 0):
            if(sqpbraid[i]<0):
                while((i+k<l) and (sqpbraid[i+k] < 0)):
                    k=k+1
                formatted = formatted[0:len(formatted)-(2*k+1)] + '{' + formatted[len(formatted)-(2*k+1):len(formatted)]
                for j in range(0,k):
                    formatted = formatted + ',' + str(sqpbraid[i + j])
                formatted = formatted + '}'
                i = i +k
            else:
                formatted = formatted + str(sqpbraid[i])
                i = i+1
        else:
            if(sqpbraid[i]<0):
                while((i+k<l) and (sqpbraid[i+k] < 0)):
                    k=k+1
                formatted = formatted[0:len(formatted)-(2*k+1)] + '{' + formatted[len(formatted)-(2*k+1):len(formatted)]
                for j in range(0,k):
                    formatted = formatted + ',' + str(sqpbraid[i + j])
                formatted = formatted + '}'
                i = i +k
            else:
                formatted = formatted + ',' + str(sqpbraid[i])
                i = i+1
    formatted = formatted + '}'
    return formatted

In [7]:
#function to format qp braid notation
def format_qpbraid(is_mirror, qpbraid):
    formatted = u''
    i = 0
    l = len(qpbraid)
    #make list of indices of rightmost generators of bands
    ends_of_bands = []
    if(is_mirror=='Y'):
        formatted = formatted + 'M{'
    else:
        formatted = formatted + '{'
        
    while((i < l)and(i < 1000)):
        if(i == 0):
            if(qpbraid[i] < 0):
                k=0
                if(not(i-1 in ends_of_bands)):
                #first test if we're in a sqp band
                    k = radius_of_conj(qpbraid,i-1,ends_of_bands)
                if(k>0):
                    #we're in a sqp band. Now check if this band is nested in a qp band
                    pivot = i-1
                    test_indices = search_list(qpbraid,invert_band(qpbraid[pivot-k:pivot+k+1]),pivot+k+1)
                    if(test_indices):                       
                        for j in test_indices:
                            test_pivot = (j + pivot + k)/2
                            if((test_pivot == int(test_pivot)) and (radius_of_conj(qpbraid,test_pivot,ends_of_bands)>=test_pivot-(pivot-k))):
                            #we are nested in a qp band
                                k = radius_of_conj(qpbraid,test_pivot,ends_of_bands)
                                pivot = test_pivot
                    if(pivot- i > 0):
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1,len(formatted)]+str(qpbraid[i])
                        for j in range(i+1, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    else:
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1,len(formatted)]+str(qpbraid[i])
                        for j in range(i+1, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    formatted = formatted + '}'
                    ends_of_bands.append(pivot + k)
                    i = pivot + k +1
                else:
                    #not in sqp band. Must be in qp band. Need to search for pivot. Take i+1 for first guess.
                    k=0
                    pivot = i+1
                    found='N'
                    iters = 0
                    while((found == 'N') and (iters <1000)):
                        if((pivot < len(qpbraid)and (qpbraid[pivot]>0))):
                            k = radius_of_conj(qpbraid,pivot,ends_of_bands)
                            if(k>=(pivot-i)):
                                #found a qp band containing original index i
                                #must now make sure we're embedded in the maximal qp band
                                test_indices = search_list(qpbraid,invert_band(qpbraid[pivot-k:pivot+k+1]),pivot+k+1)
                                if(test_indices):                       
                                    for j in test_indices:
                                        test_pivot = (j + pivot+k)/2
                                        if((test_pivot == int(test_pivot)) and (radius_of_conj(qpbraid,test_pivot,ends_of_bands)>=test_pivot-(pivot-k))):
                                        #we are nested in a qp band
                                            k = radius_of_conj(qpbraid,test_pivot,ends_of_bands)
                                            pivot = test_pivot
                                found = 'Y'
                                
                            else:
                                #keep looking to the right
                                pivot = pivot +1
                                iters = iters+1
                                
                        else:
                            iters = iters +1
                            pivot = pivot+1
                            if(pivot >len(qpbraid)-1):
                                return ''
                    formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]+str(qpbraid[i])
                    for j in range(i+1, pivot + k+1):
                        formatted = formatted + ',' + str(qpbraid[j])
                    formatted = formatted + '}'
                    ends_of_bands.append(pivot + k)
                    i = pivot + k +1
            else:
                formatted = formatted + str(qpbraid[i])
                i = i+1
        #i is nonzero case
        else:
            if(qpbraid[i] < 0):
                #first test if we're in a sqp band
                k=0
                if(not(i-1 in ends_of_bands)):
                    k = radius_of_conj(qpbraid,i-1,ends_of_bands)
                if(k>0):
                    #we're in a sqp band. Now check if this band is nested in a qp band
                    pivot = i-1
                    test_indices = search_list(qpbraid,invert_band(qpbraid[pivot-k:pivot+k+1]),pivot+k+1)
                    if(test_indices):                       
                        for j in test_indices:
                            test_pivot = (j + pivot + k)/2
                            if((test_pivot == int(test_pivot)) and (radius_of_conj(qpbraid,test_pivot,ends_of_bands)>=test_pivot-(pivot-k))):
                            #we are nested in a qp band
                                k = radius_of_conj(qpbraid,test_pivot,ends_of_bands)
                                pivot = test_pivot
                    if(pivot - i > 0):
                        if(pivot-k ==0):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                            for j in range(i+1, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        elif((pivot-k==i) and (formatted[len(formatted)-2*(i-pivot+k)-1]!=',')):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +',{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                            for j in range(i+1, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        elif((pivot-k==i) and (formatted[len(formatted)-2*(i-pivot+k)-1]==',')):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                            for j in range(i+1, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        elif(formatted[len(formatted)-2*(i-pivot+k)]!=','):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +',{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                            for j in range(i, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        else:
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                            for j in range(i, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                    else:
                        if(pivot-k ==0):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                            for j in range(i, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        elif(formatted[len(formatted)-2*(i-pivot+k)+2]!=','):
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +',{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                            for j in range(i, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                        else:
                            formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                            for j in range(i, pivot + k+1):
                                formatted = formatted + ',' + str(qpbraid[j])
                    formatted = formatted + '}'
                    ends_of_bands.append(pivot + k)
                    i = pivot + k +1
                else:
                    #not in sqp band. Must be in qp band. Need to search for pivot. Take i+1 for first guess.
                    k=0
                    pivot = i+1
                    found='N'
                    iters = 0
                    while((found == 'N') and (iters <1000)):
                        if((pivot < len(qpbraid) and (qpbraid[pivot]>0))):
                            k = radius_of_conj(qpbraid,pivot,ends_of_bands)
                            if(k>=(pivot-i)):
                                #found a qp band containing original index i
                                #must now make sure we're embedded in the maximal qp band
                                test_indices = search_list(qpbraid,invert_band(qpbraid[pivot-k:pivot+k+1]),pivot+k+1)
                                if(test_indices):                       
                                    for j in test_indices:
                                        test_pivot = (j + pivot+k)/2
                                        if((test_pivot==int(test_pivot)) and (radius_of_conj(qpbraid,test_pivot,ends_of_bands)>=test_pivot-(pivot-k))):
                                        #we are nested in a qp band
                                            k = radius_of_conj(qpbraid,test_pivot,ends_of_bands)
                                            pivot = test_pivot
                                found = 'Y'
                                
                            else:
                                #keep looking to the right
                                pivot = pivot +1
                                iters = iters+1
                                
                        else:
                            pivot = pivot+1
                            if(pivot >len(qpbraid)-1):
                                return ''
                            iters = iters +1
                    if(pivot-k ==0):
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                        for j in range(i+1, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    elif((pivot-k==i) and (formatted[len(formatted)-2*(i-pivot+k)-1]!=',')):
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +',{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                        for j in range(i+1, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    elif((pivot-k==i) and (formatted[len(formatted)-2*(i-pivot+k)-1]==',')):
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)] + str(qpbraid[i])
                        for j in range(i+1, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    elif(formatted[len(formatted)-2*(i-pivot+k)]!=','):
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +',{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                        for j in range(i, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    else:
                        formatted = formatted[0:len(formatted)-2*(i-pivot+k)+1] +'{' + formatted[len(formatted)-2*(i-pivot+k)+1:len(formatted)]
                        for j in range(i, pivot + k+1):
                            formatted = formatted + ',' + str(qpbraid[j])
                    formatted = formatted + '}'
                    ends_of_bands.append(pivot+k)
                    i = pivot + k +1
            else:
                formatted = formatted + ',' + str(qpbraid[i])
                i = i+1                
        
    formatted = formatted + "}"
    return formatted

In [8]:
#returns the largest radius of conjugacy about p
#such that this radius does not overlap with other previously defined bands
def radius_of_conj(qpbraid,pivot, ends_of_bands):
    r = 0
    while((pivot-r-1 >= 0) and (pivot+r+1 < len(qpbraid)) and (qpbraid[pivot-r-1] == -qpbraid[pivot+r+1]) and (not(pivot-r-1 in ends_of_bands))):
        r = r+1
    return r

In [9]:
#inverts a given band generator (works for any numeric list)
def invert_band(band):
    inverse = []
    i = 0
    while(i < len(band)):
        inverse.append(-band[len(band)-1-i])
        i =i+1
    return inverse

In [10]:
#returns index of beginning of first substring of l matching s, beyond index start
def search_list(l, s, start):
    k = len(s)
    i = start
    indices = []
    found = 0
    while(i < len(l)):
        if(l[i:i + k] == s):
            found = 1
            indices.append(i)
            i = i+1
        else:
            i = i+1
    if(found):
        return indices
    else:
        return 0

In [11]:
#function to clean output
def clean_output(row, is_mirror, pbraid, sqpbraid, qpbraid):
    if(pbraid):
        temp = format_pbraid(is_mirror,pbraid)
        row[7].value = temp
        row[9].value = temp
        row[10].value = temp
    elif(sqpbraid):
        temp = format_sqpbraid(is_mirror,sqpbraid)
        row[9].value = temp
        row[10].value = temp
    elif(qpbraid):
        temp = format_qpbraid(is_mirror,qpbraid)
        if(temp):
            row[10].value = temp
        else:
            print("Out of bounds error in row {0}.".format(int(row[1].value)+2))

In [22]:
#main function to clean data
row_counter = 4
mismatches = []
CLEAN = input("Do you want to clean the output? Y/N")
SAVE = 'N'
SAVE_NAME = ''
for row in ws.iter_rows(min_row = 4):
    #extract data from row
    name = row[0].value
    nr = row[1].value
    bn = row[2].value
    pb = row[3].value
    p = row[4].value
    sqp = row[5].value
    qp = row[6].value
    pn = row[7].value
    ppdn = row[8].value
    sqpn = row[9].value
    qpn = row[10].value
    
    braid = []
    pbraid = []
    sqpbraid = []
    qpbraid = []
    
    #convert data from unicode into appropriate type;
    #extract_bn() will convert data into list type;
    #obtain links from braids
    if(bn[0:2] == '{{'):
        braid = extract_single_bn(bn)
    else:
        braid = extract_bn(bn)
    if(not braid):
        print('Bad data entry in braid notation of row {0}.'.format(row_counter))
    else:
        L = Link(B(braid))
    if(L.number_of_components() > 1):
        print('The braid ' + name + ' in row {0} has {1} components.'.format(row_counter, L.number_of_components()))
    if(pb == 'Y' and pn !='exists'):
        pbraid = extract_bn(pn)
        if(not pbraid):
            print('Bad data entry in positive braid notation of row {0}.'.format(row_counter))
        else:
            Lp = Link(B(pbraid))
    elif(((pb !='N')or (pn != 'does not exist'))and((pb != 'unknown')or (pn !='existence unknown'))):
        print("Error: positivity entry error in row {0}.".format(row_counter))
    elif(sqp == 'Y' and sqpn !='exists'):
        sqpbraid = extract_bn(sqpn)
        if( not sqpbraid):
            print('Bad data entry in strongly quasipositive braid notation of row {0}.'.format(row_counter))
        else:
            Lsqp = Link(B(sqpbraid))    
    elif(((sqp !='N')or (sqpn != 'does not exist'))and((sqp != 'unknown')or (sqpn !='existence unknown'))):
        print("Error: strong quasipositivity entry error in row {0}.".format(row_counter))
    elif(qp == 'Y' and qpn !='exists'):
        qpbraid = extract_bn(qpn)
        if(not qpbraid):
            print('Bad data entry in quasipositive braid notation of row {0}.'.format(row_counter))
        else:
            Lqp = Link(B(qpbraid))        
    elif(((qp !='N')or (qpn != 'does not exist'))and((qp != 'unknown')or (qpn !='existence unknown'))and((qp !='Y')or(qpn!='exists'))):
        print("Error: quasipositivity entry error in row {0}.".format(row_counter))
        print("quasipositive is " + qp + " and quasipositive_braid_notation is " + qpn)
    
    
    #check whether any positive/qp/sqp entry is the mirror image of the given braid
    if(pbraid):
        is_mirror = check_mirror(L,Lp,row_counter,mismatches)
    elif(sqpbraid):
        is_mirror = check_mirror(L,Lsqp,row_counter,mismatches)
    elif(qpbraid):
        is_mirror = check_mirror(L,Lqp,row_counter,mismatches)
    
    
    if(CLEAN == 'Y'):
    #if there are multiple representatives, enclose in brackets.
        if((bn[0:2] == '{{')and(bn[len(bn)-2:len(bn)]=='}}')):
            row[2].value = u'['+ bn[1:len(bn)-1] + u']'
    #clean the data and modify workbook
        clean_output(row, is_mirror, pbraid,sqpbraid,qpbraid)
        
    
    
    row_counter = row_counter +1
    
SAVE = input('Do you want to save? Y/N')
if(SAVE == 'Y'):
    SAVE_NAME = input('Please enter name of file to save to. Warning: will overwrite any existing file of same name in directory.')
    wb.save(SAVE_NAME + '.xlsx')
wb.close()
print(mismatches)

Do you want to clean the output? Y/N'Y'
Error: quasipositivity entry error in row 2432.
quasipositive is N and quasipositive_braid_notation is existence unknown
Out of bounds error in row 2473.
Do you want to save? Y/N'Y'
Please enter name of file to save to. Warning: will overwrite any existing file of same name in directory.'qp_braids_update2'
[]


In [17]:
wb.close()